In [1]:
import numpy as np
from astropy import units as u
from astropy.coordinates import Angle, SkyCoord, get_constellation
from astroquery.simbad import Simbad
from astroquery.exceptions import RemoteServiceError

import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.6f}'.format)

pd.set_option('display.max_columns', None)  # Ensure all columns are shown
pd.set_option('display.max_colwidth', None)  # Display full content of each field

In [2]:
# Initial setup of Simbad
Simbad.TIMEOUT = 120  # seconds
Simbad.add_votable_fields('otype', 'flux(V)', 'id(NGC)', 'id(M)')

def query_dso_objects(df):
    for index, row in df.iterrows():
        result_table = Simbad.query_object(row['displayName'])
        if result_table is not None:
            # Convert RA / Dec to decimal degree values.
            ra_hour = Angle(result_table['RA'][0], u.hour).value
            dec_deg = Angle(result_table['DEC'][0], u.deg).value
            df.at[index, 'ra'] = ra_hour
            df.at[index, 'dec'] = dec_deg
            
            # Determine the constellation
            coord = SkyCoord(ra=ra_hour * u.hour, dec=dec_deg * u.deg)
            df.at[index, 'constellation'] = get_constellation(coord)
            
            # Set Object
            df.at[index, 'subType'] = result_table['OTYPE'][0]
            #print(f"{row['displayName']}:{df.at[index, 'subType']}")
            
            # Set Flux
            if 'FLUX_V' in result_table.colnames and not np.ma.is_masked(result_table['FLUX_V'][0]):
                df.at[index, 'magnitude'] = float(result_table['FLUX_V'][0])
            else:
                df.at[index, 'magnitude'] = pd.NA
                
            # Extract NGC and IC IDs using the correct column names
            messier_id = result_table['ID_M'][0] if 'ID_M' in result_table.colnames and result_table['ID_M'][0] else None
            ngc_id = result_table['ID_NGC'][0] if 'ID_NGC' in result_table.colnames and result_table['ID_NGC'][0] else None
            
            df.at[index, 'ngcId'] = ngc_id
            
            # Set the objectId based on priority: Messier -> NGC -> IC
            if messier_id:
                df.at[index, 'objectId'] = messier_id.replace(' ', '').lower()
            elif ngc_id:
                df.at[index, 'objectId'] = ngc_id.replace(' ', '').lower()
            else:
                df.at[index, 'objectId'] = pd.NA
        else:
            print(f"No data found for {row['displayName']}")  


In [3]:
messier_object_info = {
    "M1": {"displayName": "Crab Nebula", "subType": "Supernova Remnant"},
    "M2": {"displayName": "M2", "subType": "Globular Cluster"},
    "M3": {"displayName": "M3", "subType": "Globular Cluster"},
    "M4": {"displayName": "M4", "subType": "Globular Cluster"},
    "M5": {"displayName": "M5", "subType": "Globular Cluster"},
    "M6": {"displayName": "Butterfly Cluster", "subType": "Open Cluster"},
    "M7": {"displayName": "Ptolemy Cluster", "subType": "Open Cluster"},
    "M8": {"displayName": "Lagoon Nebula", "subType": "Emission Nebula"},
    "M9": {"displayName": "M9", "subType": "Globular Cluster"},
    "M10": {"displayName": "M10", "subType": "Globular Cluster"},
    "M11": {"displayName": "Wild Duck Cluster", "subType": "Open Cluster"},
    "M12": {"displayName": "M12", "subType": "Globular Cluster"},
    "M13": {"displayName": "Hercules Cluster", "subType": "Globular Cluster"},
    "M14": {"displayName": "M14", "subType": "Globular Cluster"},
    "M15": {"displayName": "M15", "subType": "Globular Cluster"},
    "M16": {"displayName": "Eagle Nebula", "subType": "Emission Nebula"},
    "M17": {"displayName": "Omega Nebula", "subType": "Emission Nebula"},
    "M18": {"displayName": "M18", "subType": "Open Cluster"},
    "M19": {"displayName": "M19", "subType": "Globular Cluster"},
    "M20": {"displayName": "Trifid Nebula", "subType": "Emission/Reflection Nebula"},
    "M21": {"displayName": "M21", "subType": "Open Cluster"},
    "M22": {"displayName": "M22", "subType": "Globular Cluster"},
    "M23": {"displayName": "M23", "subType": "Open Cluster"},
    "M24": {"displayName": "Sagittarius Star Cloud", "subType": "Star Cloud"},
    "M25": {"displayName": "M25", "subType": "Open Cluster"},
    "M26": {"displayName": "M26", "subType": "Open Cluster"},
    "M27": {"displayName": "Dumbbell Nebula", "subType": "Planetary Nebula"},
    "M28": {"displayName": "M28", "subType": "Globular Cluster"},
    "M29": {"displayName": "M29", "subType": "Open Cluster"},
    "M30": {"displayName": "M30", "subType": "Globular Cluster"},
    "M31": {"displayName": "Andromeda Galaxy", "subType": "Spiral Galaxy"},
    "M32": {"displayName": "M32", "subType": "Dwarf Elliptical Galaxy"},
    "M33": {"displayName": "Triangulum Galaxy", "subType": "Spiral Galaxy"},
    "M34": {"displayName": "M34", "subType": "Open Cluster"},
    "M35": {"displayName": "M35", "subType": "Open Cluster"},
    "M36": {"displayName": "M36", "subType": "Open Cluster"},
    "M37": {"displayName": "M37", "subType": "Open Cluster"},
    "M38": {"displayName": "M38", "subType": "Open Cluster"},
    "M39": {"displayName": "M39", "subType": "Open Cluster"},
    "M40": {"displayName": "Winnecke 4", "subType": "Double Star"},
    "M41": {"displayName": "M41", "subType": "Open Cluster"},
    "M42": {"displayName": "Orion Nebula", "subType": "Emission Nebula"},
    "M43": {"displayName": "De Mairan's Nebula", "subType": "Emission Nebula"},
    "M44": {"displayName": "Beehive Cluster", "subType": "Open Cluster"},
    "M45": {"displayName": "Pleiades", "subType": "Open Cluster"},
    "M46": {"displayName": "M46", "subType": "Open Cluster"},
    "M47": {"displayName": "M47", "subType": "Open Cluster"},
    "M48": {"displayName": "M48", "subType": "Open Cluster"},
    "M49": {"displayName": "M49", "subType": "Elliptical Galaxy"},
    "M50": {"displayName": "M50", "subType": "Open Cluster"},
    "M51": {"displayName": "Whirlpool Galaxy", "subType": "Spiral Galaxy"},
    "M52": {"displayName": "M52", "subType": "Open Cluster"},
    "M53": {"displayName": "M53", "subType": "Globular Cluster"},
    "M54": {"displayName": "M54", "subType": "Globular Cluster"},
    "M55": {"displayName": "M55", "subType": "Globular Cluster"},
    "M56": {"displayName": "M56", "subType": "Globular Cluster"},
    "M57": {"displayName": "Ring Nebula", "subType": "Planetary Nebula"},
    "M58": {"displayName": "M58", "subType": "Spiral Galaxy"},
    "M59": {"displayName": "M59", "subType": "Elliptical Galaxy"},
    "M60": {"displayName": "M60", "subType": "Elliptical Galaxy"},
    "M61": {"displayName": "M61", "subType": "Spiral Galaxy"},
    "M62": {"displayName": "M62", "subType": "Globular Cluster"},
    "M63": {"displayName": "Sunflower Galaxy", "subType": "Spiral Galaxy"},
    "M64": {"displayName": "Black Eye Galaxy", "subType": "Spiral Galaxy"},
    "M65": {"displayName": "Leo Triplet", "subType": "Spiral Galaxy"},
    "M66": {"displayName": "Leo Triplet", "subType": "Spiral Galaxy"},
    "M67": {"displayName": "M67", "subType": "Open Cluster"},
    "M68": {"displayName": "M68", "subType": "Globular Cluster"},
    "M69": {"displayName": "M69", "subType": "Globular Cluster"},
    "M70": {"displayName": "M70", "subType": "Globular Cluster"},
    "M71": {"displayName": "M71", "subType": "Globular Cluster"},
    "M72": {"displayName": "M72", "subType": "Globular Cluster"},
    "M73": {"displayName": "M73", "subType": "Asterism"},
    "M74": {"displayName": "Phantom Galaxy", "subType": "Spiral Galaxy"},
    "M75": {"displayName": "M75", "subType": "Globular Cluster"},
    "M76": {"displayName": "Little Dumbbell Nebula", "subType": "Planetary Nebula"},
    "M77": {"displayName": "M77", "subType": "Spiral Galaxy"},
    "M78": {"displayName": "M78", "subType": "Reflection Nebula"},
    "M79": {"displayName": "M79", "subType": "Globular Cluster"},
    "M80": {"displayName": "M80", "subType": "Globular Cluster"},
    "M81": {"displayName": "Bode's Galaxy", "subType": "Spiral Galaxy"},
    "M82": {"displayName": "Cigar Galaxy", "subType": "Starburst Galaxy"},
    "M83": {"displayName": "Southern Pinwheel Galaxy", "subType": "Spiral Galaxy"},
    "M84": {"displayName": "M84", "subType": "Elliptical Galaxy"},
    "M85": {"displayName": "M85", "subType": "Lenticular Galaxy"},
    "M86": {"displayName": "M86 (Markarian's Chain)", "subType": "Lenticular Galaxy"},
    "M87": {"displayName": "Virgo A", "subType": "Elliptical Galaxy"},
    "M88": {"displayName": "M88", "subType": "Spiral Galaxy"},
    "M89": {"displayName": "M89", "subType": "Elliptical Galaxy"},
    "M90": {"displayName": "M90", "subType": "Spiral Galaxy"},
    "M91": {"displayName": "M91", "subType": "Barred Spiral Galaxy"},
    "M92": {"displayName": "M92", "subType": "Globular Cluster"},
    "M93": {"displayName": "M93", "subType": "Open Cluster"},
    "M94": {"displayName": "M94", "subType": "Spiral Galaxy"},
    "M95": {"displayName": "M95", "subType": "Barred Spiral Galaxy"},
    "M96": {"displayName": "M96", "subType": "Spiral Galaxy"},
    "M97": {"displayName": "Owl Nebula", "subType": "Planetary Nebula"},
    "M98": {"displayName": "M98", "subType": "Spiral Galaxy"},
    "M99": {"displayName": "M99", "subType": "Spiral Galaxy"},
    "M100": {"displayName": "M100", "subType": "Spiral Galaxy"},
    "M101": {"displayName": "Pinwheel Galaxy", "subType": "Spiral Galaxy"},
    "M102": {"displayName": "Spindle Galaxy", "subType": "Lenticular Galaxy"},
    "M103": {"displayName": "M103", "subType": "Open Cluster"},
    "M104": {"displayName": "Sombrero Galaxy", "subType": "Spiral Galaxy"},
    "M105": {"displayName": "M105", "subType": "Elliptical Galaxy"},
    "M106": {"displayName": "M106", "subType": "Spiral Galaxy"},
    "M107": {"displayName": "M107", "subType": "Globular Cluster"},
    "M108": {"displayName": "M108", "subType": "Spiral Galaxy"},
    "M109": {"displayName": "M109", "subType": "Barred Spiral Galaxy"},
    "M110": {"displayName": "M110", "subType": "Dwarf Elliptical Galaxy"},
}

def create_messier_catalog():
    messier_objects = [f'M{i}' for i in range(1, 111)]
    df = pd.DataFrame(messier_objects, columns=['displayName']) 
    df['objectId'] = df['displayName'].apply(lambda x: 'm' + x[1:])

    return df

def apply_messier_info(row):
    object_id_upper = row['objectId'].upper()  # Convert objectId to uppercase to match keys in object_info
    if object_id_upper in messier_object_info:
        info = messier_object_info[object_id_upper]
        row['displayName'] = info['displayName']
        row['subType'] = info['subType']
    else:
        # Default values if objectId not found in object_info
        row['displayName'] = row['displayName']  # Keep existing displayName
        row['subType'] = 'Unknown'
    return row

subtype_to_type = {
    'HIIG': 'galaxy',
    'Supernova Remnant': 'nebula',
    'SNRemnant': 'nebula',
    'Globular Cluster': 'cluster',
    'Open Cluster': 'cluster',
    'Emission Nebula': 'nebula',
    'Emission/Reflection Nebula': 'nebula',
    'Star Cloud': 'nebula',  # Star clouds are often part of nebulous regions
    'Planetary Nebula': 'nebula',
    'Spiral Galaxy': 'galaxy',
    'Dwarf Elliptical Galaxy': 'galaxy',
    'Double Star': 'star',
    'Elliptical Galaxy': 'galaxy',
    'Asterism': 'star',
    'Reflection Nebula': 'nebula',
    'Starburst Galaxy': 'galaxy',
    'Lenticular Galaxy': 'galaxy',
    'Barred Spiral Galaxy': 'galaxy',
    'OpenCluster': 'cluster',
    'RefNeb': 'nebula',
    'ISM': 'nebula',           # Interstellar Medium, often categorized under nebulae
    'HIIReg': 'nebula',        # HII Regions are part of emission nebulae
    'PlanetaryNeb': 'nebula',  # Planetary Nebula
    'DarkNeb': 'nebula',       # Dark Nebula
    'Seyfert1': 'galaxy',      # Seyfert galaxies are active galaxies
    'Cluster*': 'cluster',     # Star clusters
    'Compact_Gr_G': 'galaxy',  # Compact Group of Galaxies
    'LINER': 'galaxy',         # Galaxy Group (Caldwell 30)
    'Bubble':'nebula'
}

messier_recommended = ['m1', 'm3', 'm5', 'm8', 'm10', 'm11', 'm12', 'm13', 
                       'm16', 'm17', 'm20','m27', 'm31', 'm33', 'm42', 'm45', 'm51', 'm57', 
                       'm63', 'm64', 'm66', 'm78', 'm81', 'm82', 'm83', 'm86', 'm94', 
                       'm97', 'm101', 'm104', 'm106']  

column_order = [
    'displayName', 'objectId', 'ra', 'dec', 'constellation', 'type', 
    'subType', 'magnitude', 'ngcId', 'recommended'
]

In [4]:
df_messier = create_messier_catalog()
query_dso_objects(df_messier)
df_messier = df_messier.apply(apply_messier_info, axis=1)
df_messier['type'] = df_messier['subType'].map(subtype_to_type)
df_messier['recommended'] = False
df_messier.loc[df_messier['objectId'].isin(messier_recommended), 'recommended'] = True
df_messier = df_messier[column_order]

In [5]:
def create_dsos():
    df = pd.DataFrame({
        'displayName': pd.Series(dtype='str'),
        'objectId': pd.Series(dtype='object'),
        'ra': pd.Series(dtype='object'),
        'dec': pd.Series(dtype='object'),
        'constellation': pd.Series(dtype='object'),
        'type': pd.Series(dtype='object'),
        'subType': pd.Series(dtype='object'),
        'magnitude': pd.Series(dtype='object'),
        'ngcId': pd.Series(dtype='object'),
        'recommended': pd.Series(dtype='bool')
    })
    
    return df


def add_dso_object(df, displayName):
    # Create a new row DataFrame with specified values and defaults to pd.NA
    new_row = pd.DataFrame({
        'displayName': [displayName],
        'objectId': [pd.NA],
        'ra': [pd.NA],
        'dec': [pd.NA],
        'constellation': [pd.NA],
        'type': [pd.NA],
        'subType': [pd.NA],
        'magnitude': [pd.NA],
        'ngcId': [pd.NA],
        'recommended': [bool(True)]
    })

    # Concatenate the new row to the DataFrame
    df = pd.concat([df, new_row], ignore_index=True)
    
    return df

def assign_missing_objectIds(df):
    # --- Open Clusters / Cluster Stars ---
    # (No metadata to augment for NGC 869 or Tau CMa)

    # --- Emission / Reflection Nebulae ---
    df.loc[df['displayName'] == "Rosette Nebula", 'objectId'] = 'ldn1625'
    df.loc[df['displayName'] == "Helix Nebula", 'objectId'] = 'ngc7293'  # optional
    df.loc[df['displayName'] == "Horsehead Nebula", 'objectId'] = 'ic434'
    df.loc[df['displayName'] == 'Wizard Nebula', 'objectId'] = 'ngc7380'
    
    df.loc[df['displayName'] == "North America Nebula", 'subType'] = 'Emission Nebula' #Simbad incorrectly puts as a Cluster*    
    
    df.loc[df['displayName'] == "Cocoon Nebula", 'objectId'] = 'ic5146'
    df.loc[df['displayName'] == "Cocoon Nebula", 'subType'] = 'Emission/Reflection Nebula' #Simbad incorrectly puts as an OpenCluster

    df.loc[df['displayName'] == "NGC 1977", 'subType'] = 'Reflection Nebula' #Simbad incorrectly puts as an OpenCluster
    df.loc[df['displayName'] == "NGC 1977", 'displayName'] = 'Running Man Nebula'

    df.loc[df['displayName'] == "NGC 1499", 'displayName'] = 'California Nebula'
    
    df.loc[df['displayName'] == "NGC 2264", 'subType'] = 'HIIReg' #Simbad incorrectly puts as an OpenCluster
    df.loc[df['displayName'] == "NGC 2264", 'displayName'] = 'Cone Nebula'

    df.loc[df['displayName'] == "NGC 2359", 'displayName'] = "Thor's Helmet"

    #REVISIT Heart is technicall Sh2-190.  IC1805 is star cluster in the center of the heart.
    df.loc[df['displayName'] == "IC 1805", 'objectId'] = 'ic1805'
    df.loc[df['displayName'] == "IC 1805", 'subType'] = 'Emission Nebula' #override
    df.loc[df['displayName'] == "IC 1805", 'displayName'] = 'Heart Nebula'

    df.loc[df['displayName'] == "IC 1848", 'objectId'] = 'ic1848'
    df.loc[df['displayName'] == "IC 1848", 'subType'] = 'HIIReg' #Simbad incorrectly puts as an OpenCluster
    df.loc[df['displayName'] == "IC 1848", 'displayName'] = 'Soul Nebula'

    df.loc[df['displayName'] == "IC 410", 'objectId'] = 'ic410'
    df.loc[df['displayName'] == "IC 410", 'subType'] = 'Emission Nebula' #Simbad incorrectly puts as an Cluster*
    df.loc[df['displayName'] == "IC 410", 'displayName'] = 'Tadpoles Nebula'

    df.loc[df['displayName'] == "Sh2-261", 'objectId'] = 'sh2-261'
    df.loc[df['displayName'] == "Sh2-261", 'displayName'] = 'Sh2-261 (Lower’s Nebula)'

    df.loc[df['displayName'] == "Sh2-112", 'objectId'] = 'sh2-112'
    df.loc[df['displayName'] == "Sh2-112", 'displayName'] = 'Sh2-112'

    df.loc[df['displayName'] == "Sh2-86", 'objectId'] = 'sh2-86'
    df.loc[df['displayName'] == "Sh2-86", 'displayName'] = 'Sh2-86'

    df.loc[df['displayName'] == "Sh2-101", 'objectId'] = 'Sh2-101'
    df.loc[df['displayName'] == "Sh2-101", 'displayName'] = 'Sh2-101 (Tulip Nebula)'

    df.loc[df['displayName'] == "Sh2-142", 'objectId'] = 'sh2-142'
    df.loc[df['displayName'] == "Sh2-142", 'displayName'] = 'Sh2-142'

    df.loc[df['displayName'] == "Sh2-171", 'objectId'] = 'sh2-171'
    df.loc[df['displayName'] == "Sh2-171", 'displayName'] = 'Sh2-171'

    df.loc[df['displayName'] == "Sh2-87", 'objectId'] = 'sh2-87'
    df.loc[df['displayName'] == "Sh2-87", 'displayName'] = 'Sh2-87'

    df.loc[df['displayName'] == "Sh2-157", 'objectId'] = 'sh2-157'
    df.loc[df['displayName'] == "Sh2-157", 'displayName'] = 'Sh2-157'

    df.loc[df['displayName'] == "Sh2-155", 'objectId'] = 'sh2-155'
    df.loc[df['displayName'] == "Sh2-155", 'displayName'] = 'Sh2-155 (Cave Nebula)'

    df.loc[df['displayName'] == "Sh2-132", 'objectId'] = 'sh2-132'
    df.loc[df['displayName'] == "Sh2-132", 'displayName'] = 'Sh2-132 (Lion Nebula)'

    df.loc[df['displayName'] == "IC 2177", 'objectId'] = 'ic2177'
    df.loc[df['displayName'] == "IC 2177", 'displayName'] = 'IC 2177 (Seagull Nebula)' 
    
    df.loc[df['displayName'] == "IC 443", 'objectId'] = 'ic443'
    df.loc[df['displayName'] == "IC 443", 'displayName'] = 'IC443 (Jellyfish Nebula)'
    
    # --- Planetary Nebulae ---
    df.loc[df['displayName'] == "Sh2-274", 'objectId'] = 'sh2-274'
    df.loc[df['displayName'] == "Sh2-274", 'displayName'] = 'Sh2-274 (Medusa Nebula)'  

    # --- Galaxies / Galaxy Groups ---
    df.loc[df['displayName'] == "NGC 2903", 'displayName'] = 'NGC 2903 (barred spiral galaxy)'

    # --- Dark Nebulae ---
    df.loc[df['displayName'] == "Barnard 59", 'objectId'] = 'barnard59' 
    df.loc[df['displayName'] == "Barnard 59", 'displayName'] = 'Barnard 59 (Head of Pipe Nebula)'

    df.loc[df['displayName'] == "Barnard 68", 'objectId'] = 'barnard68' 
    df.loc[df['displayName'] == "Barnard 68", 'displayName'] = 'Barnard 68 (Ophiuchus Globule)'

    df.loc[df['displayName'] == "Barnard 72", 'objectId'] = 'barnard72'
    df.loc[df['displayName'] == "Barnard 72", 'displayName'] = 'Barnard 72 (Snake Nebula)'

    df.loc[df['displayName'] == "Barnard 86", 'objectId'] = 'barnard86'
    df.loc[df['displayName'] == "Barnard 86", 'displayName'] = 'Barnard 86 (Ink Spot)'

    df.loc[df['displayName'] == "Barnard 92", 'objectId'] = 'barnard92'
    df.loc[df['displayName'] == "Barnard 92", 'displayName'] = 'Barnard 92 (Lagoon Region)'

    df.loc[df['displayName'] == "Barnard 93", 'objectId'] = 'barnard93'
    df.loc[df['displayName'] == "Barnard 93", 'displayName'] = 'Barnard 93 (Lagoon Region)'


In [6]:
df_dso = create_dsos()

# --- Open Clusters / Cluster Stars ---
df_dso = add_dso_object(df_dso, 'NGC 869')  # Double Cluster
df_dso = add_dso_object(df_dso, 'Tau CMa')  # Tau CMa

# --- Emission / Reflection Nebulae ---
df_dso = add_dso_object(df_dso, 'Veil Nebula')  # Supernova remnant
df_dso = add_dso_object(df_dso, 'Rosette Nebula')  # Rosette Nebula
df_dso = add_dso_object(df_dso, 'Helix Nebula')  # Helix Nebula (planetary)
df_dso = add_dso_object(df_dso, 'Horsehead Nebula')  # Horsehead Nebula
df_dso = add_dso_object(df_dso, 'Wizard Nebula')  # Wizard Nebula
df_dso = add_dso_object(df_dso, 'North America Nebula')  # North America Nebula
df_dso = add_dso_object(df_dso, 'Crescent Nebula')  # Crescent Nebula
df_dso = add_dso_object(df_dso, "Cocoon Nebula")  # Cocoon Nebula
df_dso = add_dso_object(df_dso, "NGC 1977")  # Running Man Nebula
df_dso = add_dso_object(df_dso, "NGC 1499")  # California Nebula
df_dso = add_dso_object(df_dso, "NGC 2264")  # Cone Nebula
df_dso = add_dso_object(df_dso, "NGC 2359")  # Thor's Helmet
df_dso = add_dso_object(df_dso, "IC 1805")  # Heart Nebula - REVISIT: Heart is technically SH2-190
df_dso = add_dso_object(df_dso, "IC 1848")  # Soul Nebula
df_dso = add_dso_object(df_dso, "IC 410")  # Tadpoles Nebula
df_dso = add_dso_object(df_dso, "Sh2-261")  # Lower’s Nebula
df_dso = add_dso_object(df_dso, "Sh2-112")  # Sh2-112
df_dso = add_dso_object(df_dso, "Sh2-86")  # Sh2-86
df_dso = add_dso_object(df_dso, "Sh2-101")  # Tulip Nebula
df_dso = add_dso_object(df_dso, "Sh2-142")  # Sh2-142
df_dso = add_dso_object(df_dso, "Sh2-171")  # Sh2-171
df_dso = add_dso_object(df_dso, "Sh2-87")  # Sh2-87
df_dso = add_dso_object(df_dso, "Sh2-157")  # Lobster Claw Nebula
df_dso = add_dso_object(df_dso, "Sh2-155")  # Cave Nebula
df_dso = add_dso_object(df_dso, "Sh2-132")  # Lion Nebula
df_dso = add_dso_object(df_dso, "IC 2177")  # Seagull Nebula region
df_dso = add_dso_object(df_dso, "IC 443")  # Jellyfish Nebula


# --- Planetary Nebulae ---
df_dso = add_dso_object(df_dso, "Sh2-274")  # Medusa Nebula

# --- Galaxies / Galaxy Groups ---
df_dso = add_dso_object(df_dso, "NGC 2903")  # Barred Spiral Galaxy

# --- Dark Nebulae ---
df_dso = add_dso_object(df_dso, "Barnard 59")  # Head of Pipe Nebula
df_dso = add_dso_object(df_dso, "Barnard 68")  # Ophiuchus Globule
df_dso = add_dso_object(df_dso, "Barnard 72")  # Snake Nebula
df_dso = add_dso_object(df_dso, "Barnard 86")  # Ink Spot
df_dso = add_dso_object(df_dso, "Barnard 92")  # Lagoon Region
df_dso = add_dso_object(df_dso, "Barnard 93")  # Lagoon Region


query_dso_objects(df_dso)
assign_missing_objectIds(df_dso)
df_dso['type'] = df_dso['subType'].map(subtype_to_type)

In [7]:
df = pd.concat([df_dso, df_messier], ignore_index=True)
df['id'] = 0
df.to_json('dso.json', orient='records', lines=False)

In [8]:
df_dso[0:200]

,displayName,objectId,ra,dec,constellation,type,subType,magnitude,ngcId,recommended
0,NGC 869,ngc869,2.316056,57.133889,Perseus,cluster,OpenCluster,<NA>,NGC 869,True
1,Tau CMa,ngc2362,7.311389,-24.953889,Canis Major,cluster,OpenCluster,<NA>,NGC 2362,True
2,Veil Nebula,ngc6960,20.760556,30.708333,Cygnus,nebula,ISM,<NA>,NGC 6960,True
3,Rosette Nebula,ldn1625,6.540767,4.793639,Monoceros,nebula,HIIReg,<NA>,None,True
4,Helix Nebula,ngc7293,22.494040,-20.837152,Aquarius,nebula,PlanetaryNeb,13.524000,NGC 7293,True
5,Horsehead Nebula,ic434,5.683056,-2.458333,Orion,nebula,DarkNeb,<NA>,None,True
6,Wizard Nebula,ngc7380,22.792222,58.048333,Cepheus,nebula,HIIReg,<NA>,None,True
7,North America Nebula,ngc7000,20.979722,44.330000,Cygnus,nebula,Emission Nebula,<NA>,NGC 7000,True
8,Crescent Nebula,ngc6888,20.201944,38.355000,Cygnus,nebula,ISM,<NA>,NGC 6888,True
9,Cocoon Nebula,ic5146,21.891472,47.246111,Cygnus,nebula,Emission/Reflection Nebula,<NA>,None,True


In [9]:
df_dso['displayName'].tolist()

['NGC 869',
 'Tau CMa',
 'Veil Nebula',
 'Rosette Nebula',
 'Helix Nebula',
 'Horsehead Nebula',
 'Wizard Nebula',
 'North America Nebula',
 'Crescent Nebula',
 'Cocoon Nebula',
 'Running Man Nebula',
 'California Nebula',
 'Cone Nebula',
 "Thor's Helmet",
 'Heart Nebula',
 'Soul Nebula',
 'Tadpoles Nebula',
 'Sh2-261 (Lower’s Nebula)',
 'Sh2-112',
 'Sh2-86',
 'Sh2-101 (Tulip Nebula)',
 'Sh2-142',
 'Sh2-171',
 'Sh2-87',
 'Sh2-157',
 'Sh2-155 (Cave Nebula)',
 'Sh2-132 (Lion Nebula)',
 'IC 2177 (Seagull Nebula)',
 'IC443 (Jellyfish Nebula)',
 'Sh2-274 (Medusa Nebula)',
 'NGC 2903 (barred spiral galaxy)',
 'Barnard 59 (Head of Pipe Nebula)',
 'Barnard 68 (Ophiuchus Globule)',
 'Barnard 72 (Snake Nebula)',
 'Barnard 86 (Ink Spot)',
 'Barnard 92 (Lagoon Region)',
 'Barnard 93 (Lagoon Region)']